In [1]:
import re
import requests
from lxml import html
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

In [2]:
url = 'http://www.lenta.ru'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

In [3]:
response = requests.get(url, headers)

In [4]:
dom = html.fromstring(response.text)

In [5]:
raw_news = dom.xpath("//div[@class='b-yellow-box__wrap']/div[@class='item']")

In [6]:
news = []

# Использую ссылку как _id для решения проблемы дубликатов
for new in raw_news:
    base = {}
    base['source'] = url
    base['title'] = new.xpath("./a/text()")[0].replace('\xa0', ' ')
    base['_id'] = url + str(new.xpath("./a/@href")[0])
    base['date'] = '.'.join(base['_id'].split('/')[-3:-6:-1])
    news.append(base)

In [7]:
client = MongoClient('127.0.0.1', 27017)
db = client['news_base']
lenta = db.lenta

In [8]:
dke_counter = 0
for idx, _ in enumerate(news):
    
    try:
        lenta.insert_one(news[idx])
    except dke:
        dke_counter += 1
print(f'Повторений новостей найдено: {dke_counter}')

Повторений новостей найдено: 10


In [9]:
lenta.count_documents({})

10

In [10]:
# db.drop_collection(lenta)

In [11]:
for new in lenta.find():
    pprint(new)

{'_id': 'http://www.lenta.ru/news/2021/11/04/potok_zapusk/',
 'date': '04.11.2021',
 'source': 'http://www.lenta.ru',
 'title': 'В Германии усомнились в запуске «Северного потока-2» до января 2022 '
          'года'}
{'_id': 'http://www.lenta.ru/news/2021/11/04/against_vaccine/',
 'date': '04.11.2021',
 'source': 'http://www.lenta.ru',
 'title': 'В России призвали «сдавать» антипрививочников следователям'}
{'_id': 'http://www.lenta.ru/news/2021/11/04/social_network/',
 'date': '04.11.2021',
 'source': 'http://www.lenta.ru',
 'title': 'Названы опасные для психики и интеллекта функции соцсетей'}
{'_id': 'http://www.lenta.ru/news/2021/11/04/cult/',
 'date': '04.11.2021',
 'source': 'http://www.lenta.ru',
 'title': 'Объяснена причина появления в России 1990-х культа японских машин'}
{'_id': 'http://www.lenta.ru/news/2021/11/03/antiprivivochniki/',
 'date': '03.11.2021',
 'source': 'http://www.lenta.ru',
 'title': 'В России начнут сообщать следователям об антипрививочниках'}
{'_id': 'http:/